In [2]:
import pandas as pd
import numpy as np
import io
from ipywidgets import FileUpload
from IPython.display import display

# ==========================================
# 1️⃣ DRAG & DROP UPLOAD WIDGET
# ==========================================

uploader = FileUpload(
    accept='.csv,.xlsx,.xls',
    multiple=False
)

display(uploader)

print("⬆ Drag and drop your file above")

# ==========================================
# 2️⃣ WAIT FOR FILE UPLOAD
# ==========================================

def process_file(change):
    
    if uploader.value:
        
        uploaded_file = list(uploader.value.values())[0]
        file_name = uploaded_file['metadata']['name']
        file_content = uploaded_file['content']
        
        print(f"\nFile uploaded: {file_name}")
        
        # Detect file type
        if file_name.endswith(".csv"):
            raw_df = pd.read_csv(io.BytesIO(file_content))
        elif file_name.endswith((".xlsx", ".xls")):
            raw_df = pd.read_excel(io.BytesIO(file_content))
        else:
            print("Unsupported file format")
            return
        
        print("File loaded successfully")
        
        # ==========================================
        # 3️⃣ STANDARDIZE COLUMN NAMES
        # ==========================================
        
        raw_df.columns = (
            raw_df.columns
            .str.strip()
            .str.lower()
            .str.replace(" ", "_")
        )
        
        # ==========================================
        # 4️⃣ REMOVE DUPLICATE INCIDENT ROWS
        # ==========================================
        
        disposition_col = None
        for col in raw_df.columns:
            if "disposition" in col:
                disposition_col = col
                break
        
        if disposition_col:
            raw_df[disposition_col] = raw_df[disposition_col].astype(str)
            
            before_count = len(raw_df)
            
            raw_df = raw_df[
                ~raw_df[disposition_col]
                .str.strip()
                .str.lower()
                .str.contains("duplicate incident", na=False)
            ]
            
            after_count = len(raw_df)
            print(f"Duplicate Incident rows removed: {before_count - after_count}")
        else:
            print("No disposition column found")
        
        # ==========================================
        # 5️⃣ CREATE CASE LOG
        # ==========================================
        
        possible_case_cols = [
            "incident_id",
            "incident_thread_id",
            "category_id",
            "status",
            "status_type",
            "sellerid",
            "seller_id",
            "disposition_id",
            "disposition",
            "month",
            "partner",
            "tier",
            "domain"
        ]
        
        available_case_cols = [col for col in possible_case_cols if col in raw_df.columns]
        
        case_log = raw_df[available_case_cols].copy()
        
        if "incident_id" in case_log.columns:
            case_log.rename(columns={"incident_id": "case_id"}, inplace=True)
        
        case_log = case_log.drop_duplicates(subset=["case_id"])
        
        # ==========================================
        # 6️⃣ CREATE EVENT LOG
        # ==========================================
        
        possible_event_cols = [
            "incident_id",
            "incident_date_created",
            "l1_to_l2_modified_time",
            "escalated_to_l3_date",
            "incident_date_closed"
        ]
        
        available_event_cols = [col for col in possible_event_cols if col in raw_df.columns]
        
        event_df = raw_df[available_event_cols].copy()
        
        if "incident_id" in event_df.columns:
            event_df.rename(columns={"incident_id": "case_id"}, inplace=True)
        
        for col in event_df.columns:
            if col != "case_id":
                event_df[col] = pd.to_datetime(event_df[col], errors="coerce")
        
        event_log = event_df.melt(
            id_vars=["case_id"],
            var_name="activity",
            value_name="timestamp"
        )
        
        event_log = event_log.dropna(subset=["timestamp"])
        event_log = event_log.sort_values(["case_id", "timestamp"])
        
        # ==========================================
        # 7️⃣ GENERATE VARIANT
        # ==========================================
        
        variant_df = event_log.groupby("case_id")["activity"].apply(
            lambda x: " > ".join(x)
        ).reset_index()
        
        variant_df.rename(columns={"activity": "variant"}, inplace=True)
        
        case_log = case_log.merge(variant_df, on="case_id", how="left")
        
        # ==========================================
        # 8️⃣ EXPORT FILES
        # ==========================================
        
        case_log.to_csv("final_case_log.csv", index=False)
        event_log.to_csv("final_event_log.csv", index=False)
        
        print("\n✅ Case Log and Event Log generated successfully")
        print("Files saved in current notebook directory.")

uploader.observe(process_file, names='value')

  Using cached pandas-3.0.1-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
Using cached pandas-3.0.1-cp313-cp313-win_amd64.whl (9.7 MB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)

   ---------------------------------------- 0/2 [openpyxl]
   ---------------------------------------- 0/2 [openpyxl]
   ---------------------------------------- 0/2 [openpyxl]
   ---------------------------------------- 0/2 [openpyxl]
   ---------------------------------------- 0/2 [openpyxl]
   ---------------------------------------- 0/2 [openpyxl]
   ---------------------------------------- 0/2 [openpyxl]
   ---------------------------------------- 0/2 [openpyxl]
   ---------------------------------------- 0/2 [openpyxl]
   ---------------------------------------- 0/2 [openpyxl]
   ---------------------------------------- 0/2 [openpyxl]
   ---------------------------------------- 0/2 [openpyxl]
   -----------------------------

FileUpload(value=(), accept='.csv,.xlsx,.xls', description='Upload')

⬆ Drag and drop your file above


In [4]:
!pip install pandas numpy openpyxl ipywidgets